Actividad Guiada 3 de Algoritmos de Optimizacion

Nombre: Eduardo Perea Montesino

In [4]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios
import gzip


In [5]:

# Descargar archivo comprimido
file_url = "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz"
file = "swiss42.tsp.gz"
urllib.request.urlretrieve(file_url, file_name)

# Leer contenido del archivo como bytes
with open(file, 'rb') as f:
    file_content = f.read()

# Descomprimir archivo
uncompressed_content = gzip.decompress(file_content)

# Guardar archivo descomprimido
output_file_name = "swiss42.tsp"
with open(output_file_name, 'wb') as f:
    f.write(uncompressed_content)


In [10]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(output_file_name)

#Nodos
Nodos = list(problem.get_nodes())
print('Estos son los nodos:  ', Nodos)
#Aristas
Aristas = list(problem.get_edges())

Estos son los nodos:   [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]


In [11]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
                
#dir(problem)

15

# Funcionas basicas

In [12]:
  
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
    solucion = [Nodos[0]]
    for n in Nodos[1:]:
        solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
    return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
     return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion)-1):
        distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
    return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


# BUSQUEDA ALEATORIA

In [13]:
def busqueda_aleatoria(problem, N):
    #N es el numero de iteraciones
    Nodos = list(problem.get_nodes())
  
    mejor_solucion = []
    #mejor_distancia = 10e100                         #Inicializamos con un valor alto
    mejor_distancia = float('inf')                    #Inicializamos con un valor alto
  
    for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
        solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
        distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
        if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
            mejor_solucion = solucion
            mejor_distancia = distancia
      
    print("Mejor solución:" , mejor_solucion) 
    print("Distancia     :" , mejor_distancia) 
    return mejor_solucion  

    
#Busqueda aleatoria con 5000 iteraciones    
solucion = busqueda_aleatoria(problem, 10000)    

Mejor solución: [0, 12, 10, 6, 14, 22, 30, 25, 27, 4, 5, 9, 7, 38, 8, 33, 31, 36, 15, 3, 20, 1, 28, 40, 39, 21, 24, 23, 2, 26, 11, 18, 13, 37, 17, 16, 19, 35, 29, 41, 32, 34]
Distancia     : 3654


# BUSQUEDA LOCAL

In [16]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
    mejor_solucion = []
    mejor_distancia = 10e100
    for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
        for j in range(i+1, len(solucion)):
      
            #Se genera una nueva solución intercambiando los dos nodos i,j:
            #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
            vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]   

            #Se evalua la nueva solución ...
            distancia_vecina = distancia_total(vecina, problem)         

            #... para guardarla si mejora las anteriores
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina
    return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 

nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 3654
Distancia Mejor Solucion Local: 3337


In [17]:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
    mejor_solucion = []
  
    #Generar una solucion inicial de referencia(aleatoria)
    solucion_referencia = crear_solucion(Nodos)
    mejor_distancia = distancia_total(solucion_referencia, problem)

    iteracion=0             #Un contador para saber las iteraciones que hacemos
    while(1):
        iteracion +=1         #Incrementamos el contador
        #print('#',iteracion)

        #Obtenemos la mejor vecina ...
        vecina = genera_vecina(solucion_referencia)

        #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
        distancia_vecina = distancia_total(vecina, problem)
 
        #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
        if distancia_vecina < mejor_distancia:
            #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
            mejor_solucion = vecina                   #Guarda la mejor solución encontrada
            mejor_distancia = distancia_vecina

        else:
            print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
            print("Distancia     :" , mejor_distancia) 
            return mejor_solucion
      
        solucion_referencia = vecina
 
 
sol = busqueda_local(problem)

En la iteracion  37 , la mejor solución encontrada es: [0, 7, 15, 37, 31, 20, 33, 35, 36, 17, 1, 6, 4, 3, 2, 18, 12, 11, 25, 41, 23, 10, 13, 19, 16, 14, 27, 28, 22, 38, 34, 32, 30, 29, 9, 39, 24, 40, 21, 8, 26, 5]
Distancia     : 1870


# SIMULATED ANNEALING

In [20]:
#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
      i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
      return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

    
#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
    if random.random() <  math.exp( -1*d / T)  :
        return True
    else:
        return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
    return T*0.99

In [23]:
def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura
  
    solucion_referencia = crear_solucion(Nodos)
    distancia_referencia = distancia_total(solucion_referencia, problem)
  
    mejor_solucion = []             #x* del seudocodigo
    mejor_distancia = 10e100        #F* del seudocodigo
  
  
    N=0
    while TEMPERATURA > .0001:
        N+=1
        #Genera una solución vecina
        vecina =genera_vecina_aleatorio(solucion_referencia)
    
        #Calcula su valor(distancia)
        distancia_vecina = distancia_total(vecina, problem)
      
        #Si es la mejor solución de todas se guarda(siempre!!!)
        if distancia_vecina < mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
    
        #Si la nueva vecina es mejor se cambia  
        #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
        if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
        #solucion_referencia = copy.deepcopy(vecina)
            solucion_referencia = vecina
            distancia_referencia = distancia_vecina

        #Bajamos la temperatura
        TEMPERATURA = bajar_temperatura(TEMPERATURA)
 
    print("La mejor solución encontrada es " , end="")
    print(mejor_solucion)
    print("con una distancia total de " , end="")
    print(mejor_distancia)
    return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 32, 34, 33, 20, 31, 17, 37, 15, 14, 1, 2, 28, 9, 41, 25, 11, 12, 8, 10, 13, 19, 16, 36, 35, 7, 5, 4, 6, 26, 18, 23, 40, 24, 21, 39, 29, 22, 38, 30, 27, 3]
con una distancia total de 1863


# II) COLONIA DE HORMIGAS

In [26]:
#DATOS DEL PROBLEMA
# Descargar archivo comprimido
file_url = "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz"
file = "swiss42.tsp.gz"
urllib.request.urlretrieve(file_url, file_name)

# Leer contenido del archivo como bytes
with open(file, 'rb') as f:
    file_content = f.read()

# Descomprimir archivo
uncompressed_content = gzip.decompress(file_content)

# Guardar archivo descomprimido
output_file_name = "swiss42.tsp"
with open(output_file_name, 'wb') as f:
    f.write(uncompressed_content)
    
problem = tsplib95.load(output_file_name)

#Nodos
Nodos = list(problem.get_nodes())

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
    return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion(lista de nodos)
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion)-1):
        distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
    return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [27]:
#La función 'Add_Nodo' selecciona al azar uno nodo con probabilidad uniforme
def Add_Nodo(problem, H ,T ) :
  #Mejora:Establecer una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
    Nodos = list(problem.get_nodes())
    return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H ) :
  #Incrementa segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
    for i in range(len(H)-1):
        T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
    return T

def Evaporar_Feromonas(T ):
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  #Mejora:Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
    T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
    return T

In [30]:
def hormigas(problem, N) :
    #problem = datos del problema
    #N = Número de agentes(hormigas)
    
    #Nodos
    Nodos = list(problem.get_nodes())
    #Aristas
    Aristas = list(problem.get_edges()) 
  
    #Inicializa las aristas con una cantidad inicial de feromonas:1
    #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
    T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
    #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
    Hormiga = [[0] for _ in range(N)]
  
    #Recorre cada agente construyendo la solución
    for h in range(N) :
      #Para cada agente se construye un camino
      for i in range(len(Nodos)-1) :
      
            #Elige el siguiente nodo
            Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
            Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
      T = Incrementa_Feromona(problem, T, Hormiga[h] )
    
    #Evapora Feromonas  
      T = Evaporar_Feromonas(T)

    #Seleccionamos el mejor agente
    mejor_solucion = []
    mejor_distancia = 10e100
    for h in range(N) :
        distancia_actual = distancia_total(Hormiga[h], problem)
        if distancia_actual < mejor_distancia:
            mejor_solucion = Hormiga[h]
            mejor_distancia =distancia_actual
  
  
    print(mejor_solucion)
    print(mejor_distancia)
  
  
hormigas(problem, 1000)

[0, 30, 34, 38, 39, 28, 14, 5, 7, 9, 18, 13, 15, 1, 4, 19, 11, 21, 23, 40, 32, 37, 26, 8, 41, 6, 16, 20, 12, 27, 17, 31, 35, 36, 3, 2, 33, 25, 29, 24, 10, 22]
3863
